# Recommendation Engine  
>Online judges provide a platform where many users solve problems everyday to improve their programming skills. The users can be beginners or experts in competitive programming. Some users might be good at solving specific category of problems(e.g. Greedy, Graph algorithms, Dynamic Programming etc.) while others may be beginners in the same. There can be patterns to everything, and the goal of the machine learning would be to identify these patterns and model user’s behaviour from these patterns. The goal of this challenge is to predict range of attempts a user will make to solve a given problem given user and problem details. Finding these patterns can help the programming committee, as it will help them to suggest relevant problems to solve and provide hints automatically on which users can get stuck.  

## Objective
> **The goal of this challenge is to predict range of attempts a user will make to solve a given problem given user and problem details**.

attempts_range| No. of attempts lies inside
---- | ----
1 | 1-1
2 | 2-3
3 | 4-5
4 | 6-7
5 | 8-9
6 |>=10

### source: [Analutics Vidhya](https://datahack.analyticsvidhya.com/contest/practice-problem-recommendation-engine/#ProblemStatement)

- This is a problem classification.
- 13 features
- 155295 rows
- Challenges: concatenate information, reducing information, missing values. 



## Importing libraries and modules

In [1]:
#%conda install -c conda-forge upsetplot

In [2]:
%run missing_values.ipynb #charging a local module

In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_selector

Several datasets were given let's check our first dataset.

## Data of the problems (rows= 6544, cols= 4)

problem_data.csv - This is the file containing data of the problems. It contains the following features :
key | values
----|----
problem_id | unique ID assigned to each problem
level_id | the difficulty level of the problem between ‘A’ to ‘N’
points | amount of points for the problem
tags | problem tag(s) like greedy, graphs, DFS etc.

In [4]:
problems_df = pd.read_csv("../data/raw/train/problem_data.csv")
problems_df.head()

,problem_id,level_type,points,tags
0,prob_3649,H,NaN,NaN
1,prob_6191,A,NaN,NaN
2,prob_2020,F,NaN,NaN
3,prob_313,A,500.0,"greedy,implementation"
4,prob_101,A,500.0,"constructive algorithms,greedy,math"


In [5]:
problems_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6544 entries, 0 to 6543
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   problem_id  6544 non-null   object 
 1   level_type  6411 non-null   object 
 2   points      2627 non-null   float64
 3   tags        3060 non-null   object 
dtypes: float64(1), object(3)
memory usage: 204.6+ KB


In [128]:
#problems_df_cat_columns_ = make_column_selector(dtype_include="object")(problems_df)
_ = problems_df.apply(lambda x: print(f"****** {x.name} ----{len(x.unique())}----{x.unique()} ****"))

****** problem_id ----6544----['prob_3649' 'prob_6191' 'prob_2020' ... 'prob_3269' 'prob_5071'
 'prob_1124'] ****
****** level_type ----15----['H' 'A' 'F' 'E' 'B' 'D' 'C' 'K' 'I' nan 'G' 'J' 'M' 'L' 'N'] ****
****** points ----31----[      nan  5.00e+02  5.00e+00  1.00e+03  2.00e+03  1.50e+03  2.75e+03
  1.75e+03  2.50e+03  8.00e+00  7.50e+02  2.25e+03  3.00e+03  1.25e+03
  1.00e+01 -1.00e+00  2.50e+02  5.00e+03  3.25e+03  4.00e+01  6.00e+01
  3.50e+03  3.00e+00  2.00e+01  4.00e+00  3.00e+01  1.20e+01  5.00e+01
  7.00e+01  6.00e+00  1.10e+01] ****


In [148]:
problems_df_num_columns= make_column_selector(dtype_include="number")(problems_df)
_ = problems_df[problems_df_num_columns].apply(lambda x: print(f"\n{x.name} min: {x.min()}--max: {x.max()} -- {x.value_counts()}"))


points min: -1.0--max: 5000.0 --  500.0     561
 1000.0    547
 1500.0    433
 2000.0    372
 2500.0    347
 3000.0    107
 1750.0     50
 2250.0     49
 1250.0     39
 750.0      33
 2750.0     22
 3500.0     14
 250.0      12
 30.0        7
 3250.0      5
 3.0         4
 70.0        4
 8.0         3
 50.0        3
 10.0        3
 4.0         2
 40.0        2
 60.0        1
-1.0         1
 20.0        1
 12.0        1
 5000.0      1
 5.0         1
 6.0         1
 11.0        1
Name: points, dtype: int64


There is a value of -1 let's change that value to 1

In [152]:
problems_df["points"].replace(-1,1,inplace=True)
_ = problems_df[problems_df_num_columns].apply(lambda x: print(f"\n{x.name} min: {x.min()}--max: {x.max()}"))


points min: 1.0--max: 5000.0


The columns tags seems not to be useful for the analysis, so it will be deleted. Additionally the difficulty of the problem is given by the column level_type, the columns points could be deleted.

In [6]:
problems_df.drop(columns=["tags"], inplace=True)

#### checking missing values in the column level_type

In [153]:
problems_df.missing.number_complete() # number of completed values

15582

In [121]:
problems_df.missing.number_missing() # number of missing values

4050

In [122]:
problems_df.missing.missing_variable_summary()

,variable,n_missing,n_cases,pct_missing
0,problem_id,0,6544,0.000000
1,level_type,133,6544,2.032396
2,points,3917,6544,59.856357


There is an 2% of missing values in the column points	

In [123]:
rows_with_missing_values = problems_df.missing.missing_case_summary()

In [124]:
rows_with_missing_values[rows_with_missing_values["pct_missing"]>0]

,case,n_missing,pct_missing
0,0,1,20.0
1,1,1,20.0
2,2,1,20.0
5,5,1,20.0
6,6,1,20.0
...,...,...,...
6534,6534,1,20.0
6536,6536,1,20.0
6538,6538,1,20.0
6539,6539,1,20.0


In [125]:
problems_df.missing.missing_case_table()

,n_missing_in_case,n_cases,pct_case
0,1,3854,58.893643
1,0,2592,39.608802
2,2,98,1.497555


In [129]:
(
    problems_df.sort_values(by="level_type")
    .missing
    .missing_variable_span(
        variable="points",
        span_every=440
    )
)

,span_counter,n_missing,n_complete,pct_missing,pct_complete
0,0,210,230,47.727273,52.272727
1,1,198,242,45.000000,55.000000
2,2,185,255,42.045455,57.954545
3,3,189,251,42.954545,57.045455
4,4,198,242,45.000000,55.000000
5,5,195,245,44.318182,55.681818
6,6,212,228,48.181818,51.818182
7,7,206,234,46.818182,53.181818
8,8,216,224,49.090909,50.909091
9,9,208,232,47.272727,52.727273


In [154]:
problems_df.groupby("level_type")["points"].unique()

level_type
A    [nan, 500.0, 1000.0, 750.0, 1750.0, 3000.0, 25...
B    [nan, 5.0, 1000.0, 750.0, 500.0, 1250.0, 1500....
C    [nan, 1500.0, 2000.0, 1750.0, 1250.0, 1000.0, ...
D    [nan, 2000.0, 1750.0, 2250.0, 1500.0, 500.0, 2...
E    [nan, 2750.0, 2000.0, 2500.0, 3000.0, 5000.0, ...
F    [nan, 3000.0, 2500.0, 2750.0, 3250.0, 2250.0, ...
G    [nan, 2500.0, 3500.0, 1500.0, 3000.0, 500.0, 3...
H                                [nan, 1500.0, 3500.0]
I                                                [nan]
J                                                [nan]
K                                                [nan]
L                                                [nan]
M                                                [nan]
N                                                [nan]
Name: points, dtype: object

In [155]:
problems_df.groupby("level_type").agg({"points":["max","min","mean"]})

points                     
               max     min         mean
level_type                             
A           3000.0     3.0   522.310160
B           3000.0     1.0   994.864028
C           3000.0     8.0  1504.516000
D           3000.0   500.0  2007.369615
E           5000.0   500.0  2498.188406
F           3500.0   500.0  2724.637681
G           3500.0   500.0  2989.130435
H           3500.0  1500.0  2833.333333
I              NaN     NaN          NaN
J              NaN     NaN          NaN
K              NaN     NaN          NaN
L              NaN     NaN          NaN
M              NaN     NaN          NaN
N              NaN     NaN          NaN

In [160]:
problems_df[(problems_df["level_type"].isna()==True) & (problems_df["points"].isna()==False)].sort_values(by="points")

,problem_id,level_type,points
1391,prob_4124,NaN,3.0
5346,prob_2859,NaN,3.0
4879,prob_1461,NaN,4.0
1658,prob_1031,NaN,4.0
2646,prob_1947,NaN,6.0
95,prob_4785,NaN,8.0
4989,prob_745,NaN,8.0
5272,prob_2011,NaN,10.0
3448,prob_5670,NaN,11.0
2198,prob_2345,NaN,12.0


Observing the information above we can could said that the problems could be categorize in the level_type A - B- C - D

In [31]:
#problems_df[problems_df["problem_id"] == "prob_4124"]
#problems_df.loc[1391, "level_type"] = "A"
def replace_null_values(df, indexes):
    for index in indexes:
        points = df.loc[index, "points"]
        if points < 8:
            df.loc[index, "level_type"] = "A"
        elif points < 500:
            df.loc[index, "level_type"] = "C"
        else:
            df.loc[index, "level_type"] = "D"

In [29]:
indexes = list((problems_df[(problems_df["level_type"].isna()==True) & (problems_df["points"].isna()==False)].sort_values(by="points")).index)

In [32]:
replace_null_values(problems_df, indexes)

In [34]:
problems_df[(problems_df["level_type"].isna()==True)].sort_values(by="points")

,problem_id,level_type,points
28,prob_3284,NaN,NaN
41,prob_4968,NaN,NaN
42,prob_704,NaN,NaN
88,prob_145,NaN,NaN
115,prob_1535,NaN,NaN
...,...,...,...
5885,prob_6179,NaN,NaN
5976,prob_1221,NaN,NaN
6041,prob_6467,NaN,NaN
6391,prob_2702,NaN,NaN


In [58]:
problems_df.drop(columns="points", inplace=True)

Until now we have used the column of "points" to estimate the level of a problem. But now due to the quantity of missing values this column will be delete. It can be said that the "level_type" is a feature that describe better a problem.

There is still 98 problems without level_type. Let's concatenate the data frames and look for the time that takes a problem to be solved

## Submissions

submissions_data.csv - This is the file containing data of the submissions. It contains the following features :
key | value
----|----
user_id | Unique ID assigned to each user
problem_id | unique ID assigned to each problem
attemps_range | column to be predicted

In [35]:
submissions_df = pd.read_csv("../data/raw/train/train_submissions.csv")
submissions_df.head()

,user_id,problem_id,attempts_range
0,user_232,prob_6507,1
1,user_3568,prob_2994,3
2,user_1600,prob_5071,1
3,user_2256,prob_703,1
4,user_2321,prob_356,1


In [83]:
submissions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155295 entries, 0 to 155294
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   user_id         155295 non-null  object
 1   problem_id      155295 non-null  object
 2   attempts_range  155295 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.6+ MB


In [84]:
_ = submissions_df.apply(lambda x: print(f"****** {x.name} ----{len(x.unique())}----{x.unique()} ****"))

****** user_id ----3529----['user_232' 'user_3568' 'user_1600' ... 'user_3212' 'user_208' 'user_3306'] ****
****** problem_id ----5776----['prob_6507' 'prob_2994' 'prob_5071' ... 'prob_4986' 'prob_1486'
 'prob_1016'] ****
****** attempts_range ----6----[1 3 2 4 5 6] ****


The information in the submissions data set is consistent and there are not missing values.

## Users df

user_data.csv. This is the file containing data of users. It contains the following features :
key | value
----|----
user_id | unique ID assigned to each user
submission_count | total number of user submissions
problem_solved | total number of accepted user submissions
contribution | user contribution to the judge
country | location of user
follower_count | amount of users who have this user in followers
last_online_time_seconds | time when user was last seen online
max_rating | maximum rating of user
rating | rating of user
rank | can be one of ‘beginner’ ,’intermediate’ , ‘advanced’, ‘expert’
registration_time_seconds | time when user was registered

In [36]:
users_df = pd.read_csv("../data/raw/train/user_data.csv")
users_df.head()

,user_id,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds
0,user_3311,47,40,0,NaN,4,1504111645,348.337,330.849,intermediate,1466686436
1,user_3028,63,52,0,India,17,1498998165,405.677,339.450,intermediate,1441893325
2,user_2268,226,203,-8,Egypt,24,1505566052,307.339,284.404,beginner,1454267603
3,user_480,611,490,1,Ukraine,94,1505257499,525.803,471.330,advanced,1350720417
4,user_650,504,479,12,Russia,4,1496613433,548.739,486.525,advanced,1395560498


In [50]:
users_df.duplicated().sum()

0

In [76]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3571 entries, 0 to 3570
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    3571 non-null   object 
 1   submission_count           3571 non-null   int64  
 2   problem_solved             3571 non-null   int64  
 3   contribution               3571 non-null   int64  
 4   country                    2418 non-null   object 
 5   follower_count             3571 non-null   int64  
 6   last_online_time_seconds   3571 non-null   int64  
 7   max_rating                 3571 non-null   float64
 8   rating                     3571 non-null   float64
 9   rank                       3571 non-null   object 
 10  registration_time_seconds  3571 non-null   int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 307.0+ KB


In [79]:
user_df_num_col = make_column_selector(dtype_include="number")(users_df)
_ = users_df[user_df_num_col].apply(lambda x: print(f"\n{x.name} ----{len(x)}----max:{x.max()}---min:{x.min()}"))


submission_count ----3571----max:4570---min:1

problem_solved ----3571----max:4476---min:0

contribution ----3571----max:171---min:-64

follower_count ----3571----max:10575---min:0

last_online_time_seconds ----3571----max:1505595074---min:1484237387

max_rating ----3571----max:983.085---min:303.899

rating ----3571----max:911.124---min:0.0

registration_time_seconds ----3571----max:1484236125---min:1264761387


In [15]:
len(users_df["user_id"].unique())

3571

In [82]:
user_df_cat_col = make_column_selector(dtype_include="object")(users_df)
_ = users_df[user_df_cat_col].apply(lambda x: print(f"\n{x.name} ----{len(x.unique())}----{x.unique()}"))


user_id ----3571----['user_3311' 'user_3028' 'user_2268' ... 'user_1929' 'user_2772'
 'user_2179']

country ----80----[nan 'India' 'Egypt' 'Ukraine' 'Russia' 'China' 'Indonesia' 'Syria'
 'Poland' 'Vietnam' 'Japan' 'Colombia' 'Singapore' 'Bangladesh' 'Brazil'
 'Thailand' 'South Korea' 'Tajikistan' 'Croatia' 'Italy' 'Turkmenistan'
 'Cuba' 'Taiwan' 'Iran' 'Kazakhstan' 'Georgia' 'Bolivia' 'Belarus'
 'Germany' 'Estonia' 'United States' 'Canada' 'Moldova' 'Mongolia'
 'Tunisia' 'Armenia' 'Argentina' 'Uzbekistan' 'Kyrgyzstan' 'Romania'
 'North Korea' 'Hong Kong' 'Jordan' 'Austria' 'Serbia' 'Peru' 'Finland'
 'Switzerland' 'South Africa' 'Mexico' 'Costa Rica' 'Israel' 'Belgium'
 'United Kingdom' 'Spain' 'Netherlands' 'Slovakia' 'Czechia' 'Azerbaijan'
 'Norway' 'Australia' 'Bosnia and Herzegovina' 'Malaysia' 'Philippines'
 'Macedonia' 'Lithuania' 'Bulgaria' 'Morocco' 'Hungary' 'Chile'
 'Venezuela' 'Haiti' 'Iceland' 'Trinidad and Tobago' 'France'
 'Christmas Island' 'Swaziland' 'Laos' 'Latvia' 'L

In [39]:
user_without_country = list(users_df[users_df["country"].isna() == True]["user_id"].unique())
user_with_country =  list(users_df[users_df["country"].isna() == False]["user_id"].unique())
[i for i in user_without_country if i in user_with_country]

[]

There is no way to determine the country to which a users belongs to, so, a new category labeled as "Missing" will be created.

In [40]:
users_df["country"] = users_df["country"].fillna("Missing")

In [41]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3571 entries, 0 to 3570
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    3571 non-null   object 
 1   submission_count           3571 non-null   int64  
 2   problem_solved             3571 non-null   int64  
 3   contribution               3571 non-null   int64  
 4   country                    3571 non-null   object 
 5   follower_count             3571 non-null   int64  
 6   last_online_time_seconds   3571 non-null   int64  
 7   max_rating                 3571 non-null   float64
 8   rating                     3571 non-null   float64
 9   rank                       3571 non-null   object 
 10  registration_time_seconds  3571 non-null   int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 307.0+ KB


The information of users is consistent and there is not missing information.

## Merge

In [61]:
df = users_df.merge(submissions_df, left_on="user_id", how="inner",right_on="user_id")
df.head()

,user_id,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds,problem_id,attempts_range
0,user_3311,47,40,0,Missing,4,1504111645,348.337,330.849,intermediate,1466686436,prob_75,1
1,user_3311,47,40,0,Missing,4,1504111645,348.337,330.849,intermediate,1466686436,prob_1481,1
2,user_3311,47,40,0,Missing,4,1504111645,348.337,330.849,intermediate,1466686436,prob_3474,1
3,user_3311,47,40,0,Missing,4,1504111645,348.337,330.849,intermediate,1466686436,prob_334,1
4,user_3311,47,40,0,Missing,4,1504111645,348.337,330.849,intermediate,1466686436,prob_3508,1


In [51]:
problems_df.columns

Index(['problem_id', 'level_type', 'points'], dtype='object')

In [62]:
df = df.merge(problems_df, on="problem_id",how="inner")

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155295 entries, 0 to 155294
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   user_id                    155295 non-null  object 
 1   submission_count           155295 non-null  int64  
 2   problem_solved             155295 non-null  int64  
 3   contribution               155295 non-null  int64  
 4   country                    155295 non-null  object 
 5   follower_count             155295 non-null  int64  
 6   last_online_time_seconds   155295 non-null  int64  
 7   max_rating                 155295 non-null  float64
 8   rating                     155295 non-null  float64
 9   rank                       155295 non-null  object 
 10  registration_time_seconds  155295 non-null  int64  
 11  problem_id                 155295 non-null  object 
 12  attempts_range             155295 non-null  int64  
 13  level_type                 15

In [66]:
df.groupby("level_type").agg({"attempts_range":["max", "min","mean"]})

attempts_range              
                      max min      mean
level_type                             
A                       6   1  1.534509
B                       6   1  1.762735
C                       6   1  1.960191
D                       6   1  2.020094
E                       6   1  2.163628
F                       6   1  2.136756
G                       6   1  1.913212
H                       6   1  1.827703
I                       6   1  1.829319
J                       6   1  1.802956
K                       6   1  1.783926
L                       6   1  1.804167
M                       6   1  1.590308
N                       4   1  1.603448

Let's create a new category to label the missing values in the col level_type. 

In [67]:
df["level_type"] = df["level_type"].fillna("O")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155295 entries, 0 to 155294
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   user_id                    155295 non-null  object 
 1   submission_count           155295 non-null  int64  
 2   problem_solved             155295 non-null  int64  
 3   contribution               155295 non-null  int64  
 4   country                    155295 non-null  object 
 5   follower_count             155295 non-null  int64  
 6   last_online_time_seconds   155295 non-null  int64  
 7   max_rating                 155295 non-null  float64
 8   rating                     155295 non-null  float64
 9   rank                       155295 non-null  object 
 10  registration_time_seconds  155295 non-null  int64  
 11  problem_id                 155295 non-null  object 
 12  attempts_range             155295 non-null  int64  
 13  level_type                 15

Ok, the data set is ready to visual analysis.

In [68]:
df.to_csv(path_or_buf="../data/processed/df_cleaned.csv", index=False)